<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Getting-your-data" data-toc-modified-id="Getting-your-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Getting your data</a></span><ul class="toc-item"><li><span><a href="#Use-the-script-download_facebook.py-in-src/-to-download-the-data,-then-locate-the-csv-file." data-toc-modified-id="Use-the-script-download_facebook.py-in-src/-to-download-the-data,-then-locate-the-csv-file.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Use the script download_facebook.py in src/ to download the data, then locate the csv file.</a></span></li><li><span><a href="#Manipulating-dates" data-toc-modified-id="Manipulating-dates-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Manipulating dates</a></span></li><li><span><a href="#Your-stats" data-toc-modified-id="Your-stats-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Your stats</a></span><ul class="toc-item"><li><span><a href="#Time-spent-using-Facebook" data-toc-modified-id="Time-spent-using-Facebook-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Time spent using Facebook</a></span></li><li><span><a href="#Time-spent-watching-ads" data-toc-modified-id="Time-spent-watching-ads-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Time spent watching ads</a></span></li><li><span><a href="#Most-seen-sources" data-toc-modified-id="Most-seen-sources-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Most seen sources</a></span></li></ul></li><li><span><a href="#Exploring-with-altair-viz-tools" data-toc-modified-id="Exploring-with-altair-viz-tools-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Exploring with altair viz tools</a></span></li><li><span><a href="#Creating-wordclouds-with-the-text-of-the-posts" data-toc-modified-id="Creating-wordclouds-with-the-text-of-the-posts-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Creating wordclouds with the text of the posts</a></span></li></ul></li></ul></div>

# Getting your data

To run any script and to query the API in general, you will need a token. A code is generated every time you install the facebook.tracking.exposed

You can use the test one or enter you own. Read this if you don't know how to get your token: link.

## Use the script download_facebook.py in src/ to download the data, then locate the csv file.

In [ ]:
summary_path = '../sample_data/users/user_a.csv'

Import the necessary libraries. In this example we commented out the hierarchical configuration used to call scripts from the command line.

In [ ]:
import sys, os
sys.path.insert(0, '../')

from src import tools

import pandas as pd
# from src.lib.config import config
import datetime

print('Done!')

Now, we read the csv downloaded with facebook_download.py, remember that you can choose the amount of entries to retrieve by using the parameters --amount and --skip.

In [ ]:
df = pd.read_csv(summary_path)
print('Done!')

This is how the data looks like:

In [ ]:
from IPython.display import display

display(df.head())

## Manipulating dates

Now you can check the timeframe of the data you pulled.

In [ ]:

df = tools.setDatetimeIndex(df)
maxDate = df.index.max()
minDate = df.index.min()
print('Information for timeframe: '+str(minDate)[:-6]+' to '+str(maxDate)[:-6])

OPTIONALLY, you can also cut it to get, in this example, the last 24 hours only.

In [ ]:
# start = maxDate-datetime.timedelta(days=1)
# end = maxDate
# df = tools.setTimeframe(df, str(start), str(end))
# print('From '+str(start)+' to '+str(end)+'\n')

## Your stats

### Time spent using Facebook

You can get useful insights for yourself, for example you can estimate the you time spent of facebook during that timeframe.

In [ ]:
timelines = df.timeline.unique()
total = pd.to_timedelta(0)

for t in timelines:
    ndf = tools.filter(t, df=df, what='timeline', kind='or')
    timespent = ndf.index.max() - ndf.index.min()
    total += timespent
    
print('Time spent on Facebook in this timeframe: '+str(total))

### Time spent watching ads

In [ ]:
nature = df.nature.value_counts()

try:
    percentage = str((nature.sponsored/nature.organic)*100)[:-12]
except:
    nature['sponsored'] = 0
    percentage = str((nature.sponsored/nature.organic)*100)
    
print(percentage+'% of the posts are sponsored posts.')

timeads = (total.seconds)*(nature.sponsored/nature.organic)
print('You spent an estimate of '+str(datetime.timedelta(seconds=(timeads)))[:-7]+' watching ads on Facebook.')

### Most seen sources

You can also check which are the top news that are informing you.

In [ ]:
n = 5
top = df.source.value_counts().nlargest(n)
print('Top '+str(n)+' sources of information are: \n'+top.to_string())

Of course, you can display this data graphically. (Run field twice if it doesn't work).

In [ ]:
%matplotlib inline

top.plot.pie(autopct='%.2f', fontsize=13, figsize=(6, 6))

## Exploring with altair viz tools

Exploring seen posts. You might need to trim the data first.

In [ ]:
import altair as alt

# for the notebook only (not for JupyterLab) run this command once per session
alt.renderers.enable('notebook')
data = df[['']]
alt.Chart(df).mark_circle().encode(
    x='impressionTime:T',
    y='LIKE:Q',
    color='source:N'
).interactive()

In [ ]:
df['count'] = df.groupby('postId')['postId'].transform('count')

alt.Chart(df).transform_calculate(
    url='https://www.facebook.com' + alt.datum.permaLink
).mark_circle().encode(
    y='count:Q',
    x='average(impressionOrder):Q',
    color='source:N',
#     size='LIKE:Q',
    href='url:N',
    tooltip=['source:N', 'url:N']
).interactive()

## Creating wordclouds with the text of the posts

In [ ]:
#import necessary modules
import matplotlib
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from stop_words import get_stop_words

matplotlib.rcParams['figure.figsize'] = [20, 10]

# get the stopwords for your language
stop_words = get_stop_words('es') + get_stop_words('it')+get_stop_words('nl')+get_stop_words('en')

# define the function 
def generate_wordcloud(text):
    wordcloud = WordCloud(font_path='src/fonts/DejaVuSans.ttf',
                          relative_scaling = 1.0,
                          stopwords = stop_words,  # set or space-separated string
                          width=2000,
                          height=1000
                          ).generate(text)
    plt.imshow(wordcloud)
    plt.figsize=(20,10)
    plt.axis("off")
    plt.show()
    
text = df.texts.str.join(sep='').reset_index()
# .str().join(sep=',').reset_index()
text.columns = ['date', 'words']
text = text.words.str.cat(sep=' ')

generate_wordcloud(text)